In [5]:
from pymongo import MongoClient
import time
import operator
import datetime
import re

client = MongoClient()
db = client['twitter']
te = db['prueba2']
dia_inicio = datetime.datetime.now()
dia_inicio = datetime.datetime(dia_inicio.year, dia_inicio.month, dia_inicio.day -1)
print dia_inicio

2017-04-08 00:00:00


In [6]:
orig = te.find_one({'created_at': {"$gt" : dia_inicio}}, sort=[("created_at", 1)])["created_at"]
fin = te.find_one({'created_at': {"$gt" : dia_inicio}}, sort=[("created_at", -1)])["created_at"]
intervalo = 15
l= []
#trunco el a a minutos
a =  datetime.datetime(orig.year, orig.month, orig.day, orig.hour, orig.minute)
while a < fin: 
    sl = []
    b = a + datetime.timedelta(minutes=intervalo)
    rl = te.find({'$and': [{'created_at': {"$gte" : a}} ,{'created_at': {"$lt" : b}} ]})
    r = rl.count()
    if r >0:
        #rh = te.find({'$and': [{'text':{"$regex" : '#'}} ,{'created_at': {"$gte" : a}} ,{'created_at': {"$lt" : b}} ]}).count()
        #rhl = te.find({'$and': [{'text':{"$regex" : '#'}} ,{'created_at': {"$gte" : a}} ,{'created_at': {"$lt" : b}} ]})
        #rw = te.find({'$and': [{'text':{"$regex" : 'http'}} ,{'created_at': {"$gte" : a}} ,{'created_at': {"$lt" : b}} ]}).count()
        #rh2 = [tw for tw in r if tw['entities']['hastags']]
        #rw2 = [tw for tw in r if len(tw['entities']['urls'])>0   ]
        rw = len(filter(lambda a: len(a['entities']['urls']) >0, rl))
        rl.rewind()
        rh = len(filter(lambda a: len(a['entities']['hashtags']) >0, rl))
        sl.append(a)
        sl.append(r)
        sl.append(rh)
        sl.append(rw)
        l.append(sl)
    a=b 


In [7]:
ls = sorted(l, key=operator.itemgetter(0))
for a in ls: 
    print str(a[0]) + ' ====> ' + str(a[1]) + ' ' + "{0:.2f}".format((a[2]/float(a[1]) * 100)) + '% ' + "{0:.2f}".format((a[3]/float(a[1]) * 100)) + '%'
    
    str(a[2]/float(a[1]) * 100)

2017-04-09 09:11:00 ====> 820 16.59% 82.80%
2017-04-09 09:26:00 ====> 814 17.94% 85.38%
2017-04-09 09:41:00 ====> 712 17.84% 82.30%
2017-04-09 09:56:00 ====> 758 16.62% 83.25%
2017-04-09 10:11:00 ====> 729 17.56% 83.54%
2017-04-09 10:26:00 ====> 775 17.29% 83.74%
2017-04-09 10:41:00 ====> 731 16.96% 87.96%
2017-04-09 10:56:00 ====> 757 15.19% 84.68%
2017-04-09 11:11:00 ====> 800 16.25% 87.38%
2017-04-09 11:26:00 ====> 721 16.64% 82.11%
2017-04-09 11:41:00 ====> 793 29.38% 85.50%
2017-04-09 11:56:00 ====> 841 27.59% 82.64%
2017-04-09 12:11:00 ====> 802 18.95% 82.29%
2017-04-09 12:26:00 ====> 775 21.68% 85.55%
2017-04-09 12:41:00 ====> 745 14.09% 84.56%
2017-04-09 12:56:00 ====> 840 15.36% 82.86%
2017-04-09 13:11:00 ====> 824 17.72% 83.13%
2017-04-09 13:26:00 ====> 845 19.17% 85.09%
2017-04-09 13:41:00 ====> 879 14.56% 85.78%
2017-04-09 13:56:00 ====> 903 17.72% 88.26%
2017-04-09 14:11:00 ====> 836 14.35% 84.93%
2017-04-09 14:26:00 ====> 850 17.53% 84.71%
2017-04-09 14:41:00 ====> 793 14

In [10]:
## get url Real-- no lo usamos, esta en entities
import urllib2

class HeadRequest(urllib2.Request):
    def get_method(self): return "HEAD"
    
def get_real(url):
    res = urllib2.urlopen(HeadRequest(url))
    return res.geturl()

In [11]:
###Cogemos las urls de un intervalo
def get_urls(init, size):
    d = {}
    end = init + datetime.timedelta(minutes=size)
    rw = te.find({'$and': [{'text':{"$regex" : 'http'}} ,{'created_at': {"$gte" : init}} ,{'created_at': {"$lt" : end}} ]})
    for t in rw: 
        #web = re.search("(?P<url>https?://[^\s]+)", t['text']).group("url") 
        #print get_real(web)
        for w in t['entities']['urls']:
            if w['expanded_url'] in d:
                d[w['expanded_url']] = d[w['expanded_url']] + 1
            else:
                d[w['expanded_url']] =1
    
    return d
init = datetime.datetime(2017, 1, 22, 0, 0)
u = get_urls(init, 3600)

In [12]:
sorted(u.items(), key=operator.itemgetter(1), reverse=True)
    


[]

In [13]:
###Creamos rangos y cogemos lo más visitado
d = {}
for a in te.find({'$and': [{'text':{"$regex" : '#'}} ,{'created_at': {"$gt" : dia_inicio}} ]},  { 'text': 1}):
	s = a['text']
	for ht in  [b.strip('#') for b in s.split() if b.startswith("#")]:
		if ht not in d:
			d[ht] = 1
		else:
			d[ht] = d[ht] + 1
trending = sorted(d.items(), key=operator.itemgetter(1))
print trending


[(u'', 1), (u'ParigiRoubaix', 1), (u'MarcaEspa\xf1a:', 1), (u'siria\xbb', 1), (u'esINT', 1), (u'Venecia', 1), (u'Durant;', 1), (u'Honda', 1), (u'madera', 1), (u'CentroMadrid', 1), (u'RadiosUsadas', 1), (u'Andalucia', 1), (u'EUETS', 1), (u'Jerusal\xe9n', 1), (u'Marcas', 1), (u'solidaridad', 1), (u'Beisbol', 1), (u'SNA', 1), (u'yihadistas?', 1), (u'rmadrid', 1), (u'CD9ViveEnSuburbia', 1), (u'Iron\xeda', 1), (u'SacaElJugoATusRedes:5', 1), (u'MAN', 1), (u'Cibermentores:', 1), (u'noalaguerra', 1), (u'OriolBroggi', 1), (u'igualdad,', 1), (u'corrupci\xf3n', 1), (u'Amazon', 1), (u'lacasadelosnegocios', 1), (u'Maduro.', 1), (u'TAKPOOL', 1), (u'GerardoVera', 1), (u'Aranjuez', 1), (u'bulto', 1), (u'cuidaelplaneta', 1), (u'StopCaza\u2026', 1), (u'despilfarro', 1), (u'Bauman:', 1), (u'DigitalMarketing', 1), (u'cuota', 1), (u'Health', 1), (u'POPULAR', 1), (u'LeyViviendaPah', 1), (u'Getafe', 1), (u'Empleo', 1), (u'FerrariLand!', 1), (u'Mejor2', 1), (u'MARCAQuizSportshj.', 1), (u'LaConcertadaSegrega',